In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# Function to parse each line of the file
def parse_line(line):
    # Split the line by commas for the titles or by spaces for the data
    parts = line.strip().split(',') if ',' in line else line.strip().split()
    return parts

# Specify the names of the files to search for
target_files = ['flow_cut_to_failure.vtk', 'transport_cut_to_failure.vtk']

n = 600
# Arrays to store the read data
h2aq = np.zeros([n+1, 1])
h2saq = np.zeros([n+1, 1])
h2g = np.zeros([n+1, 1])

In [3]:
for i in range(1,n+1):
    # Get the current directory
    current_dir = './../Goe/run_' + str(i) + '/'

    # Search for the target files in the current directory
    for file in os.listdir(current_dir):
        if file in target_files:
            print("Failure found in folder:", current_dir)
            break  # Exit the loop if either file is found
        else:
            read_folder = True
            
    if not os.path.exists('./../Goe/run_' + str(i) + '/pflotran_h2_NR-012.vtk'):
            print(f"File does not exist: {'./../Goe/run_' + str(i) + '/pflotran_h2_NR-012.vtk'}")
            continue
    else:
        file_path_h2 = './../Goe/run_' + str(i) + '/pflotran_h2_NR-012.vtk'
        #============
        # Read H2(aq)
        # ============
        # Initialize an empty list to store the numbers
        numbers_list_h2 = []
        numbers_list_sat = []

        # Define a flag to identify when to start reading numbers
        start_reading_h2 = False
        start_reading_sat = False

        # Open the file and read line by line
        with open(file_path_h2, 'r') as file:
            for line in file:
                # Check if the line contains the title indicating the start of numbers
                if 'SCALARS Total_H2(aq)' in line:
                    start_reading_h2 = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_h2:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_h2 = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_h2.extend([float(num) for num in line.split()])

                if 'SCALARS Liquid_Saturation' in line:
                    start_reading_sat = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_sat:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_sat = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_sat.extend([float(num) for num in line.split()])

        # Convert the list to a numpy array
        numbers_array_h2_all = np.array(numbers_list_h2)
        numbers_array_h2 = numbers_array_h2_all[numbers_array_h2_all <= 1.0]
        numbers_array_sat_all = np.array(numbers_list_sat)
        numbers_array_sat = numbers_array_sat_all[numbers_array_h2_all <= 1.0]

        cell_vol_gr = 16*16*20*0.2*1000

        # Display the first few numbers to verify
        cell_h2 = cell_vol_gr*numbers_array_sat*numbers_array_h2
        total_h2 = np.sum(cell_h2)

        # print("Total H2(aq) is " + str(total_h2) + " moles")
        h2aq[i, 0] = total_h2

        # ============
        # Read H2S(aq)
         # ============
        file_path_h2s = './../Goe/run_' + str(i) + '/pflotran_h2_NR-012.vtk'
        # Initialize an empty list to store the numbers
        numbers_list_h2s = []
        numbers_list_sat = []

        # Define a flag to identify when to start reading numbers
        start_reading_h2s = False
        start_reading_sat = False

        # Open the file and read line by line
        with open(file_path_h2s, 'r') as file:
            for line in file:
                # Check if the line contains the title indicating the start of numbers
                if 'SCALARS H2S(aq)' in line:
                    start_reading_h2s = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_h2s:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_h2s = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_h2s.extend([float(num) for num in line.split()])

                if 'SCALARS Liquid_Saturation' in line:
                    start_reading_sat = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_sat:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_sat = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_sat.extend([float(num) for num in line.split()])

        # Convert the list to a numpy array
        numbers_array_h2s_all = np.array(numbers_list_h2s)
        numbers_array_h2s = numbers_array_h2s_all[numbers_array_h2s_all <= 1.0]
        numbers_array_sat_all = np.array(numbers_list_sat)
        numbers_array_sat = numbers_array_sat_all[numbers_array_h2s_all <= 1.0]

        cell_vol_gr = 16*16*20*0.2*1000

        # Display the first few numbers to verify
        cell_h2s = cell_vol_gr*numbers_array_sat*numbers_array_h2s
        total_h2s = np.sum(cell_h2s)

        # print("Total H2S(aq) is: " + str(total_h2s) + " moles")
        h2saq[i, 0] = total_h2s

        # ==========
        # Read H2(g)
        # ==========
        # Define the path to your .dat file
        file_path_h2g = './../Goe/run_' + str(i) + '/pflotran_h2_NR-mas.dat'

        # Read the file and parse each line
        with open(file_path_h2g, 'r') as file:
            lines = file.readlines()

        # Parse the column titles
        column_titles = parse_line(lines[0])

        # Initialize a list to hold the parsed data
        data = []

        # Parse each line of data
        for line in lines[1:]:  # Skip the first line with column titles
            parsed_line = parse_line(line)
            if parsed_line:  # Ensure the line is not empty
                data.append(parsed_line)

        # Create a DataFrame from the parsed data
        # The length of the longest row is used to ensure all rows have the same length
        max_length = max(len(row) for row in data)
        df = pd.DataFrame(data, columns=column_titles[:max_length])

        df.columns = df.columns.str.strip('"')
        last_number_global_h2 = df["Global H2(g) [mol]"].iloc[-1]
        last_number_global_h2 = pd.to_numeric(last_number_global_h2, errors='coerce')
        h2g[i, 0] = last_number_global_h2
        # print("The H2(g) [mol] after 12M is: ", last_number_global_h2)

In [4]:
print(h2aq)
print(h2saq)
print(h2g)

np.savetxt('./H2AQ_NR.txt', h2aq, fmt='%d', delimiter=', ')
np.savetxt('./H2SAQ_NR.txt', h2saq, fmt='%d', delimiter=', ')
np.savetxt('./H2G_NR.txt', h2g, fmt='%d', delimiter=', ')

[[0.00000000e+00]
 [1.37635826e+09]
 [5.97021155e+08]
 [1.67054507e+09]
 [3.85338213e+08]
 [1.06144114e+09]
 [1.51514967e+09]
 [1.46904284e+09]
 [6.44675499e+08]
 [1.65544457e+09]
 [5.13496234e+08]
 [1.71009975e+09]
 [1.11575057e+09]
 [3.37984773e+08]
 [1.40587588e+09]
 [3.61948586e+08]
 [9.61641853e+08]
 [6.70718787e+08]
 [5.05915542e+08]
 [8.02249779e+08]
 [1.67014907e+09]
 [8.03954717e+08]
 [3.67970264e+08]
 [2.72616884e+08]
 [1.44993407e+09]
 [1.31494012e+09]
 [8.60386411e+08]
 [2.21069699e+08]
 [6.97492613e+08]
 [1.38197986e+09]
 [3.09061864e+08]
 [1.06325655e+09]
 [7.21580928e+08]
 [4.87880080e+08]
 [1.04748143e+09]
 [4.41492499e+08]
 [1.46935921e+09]
 [2.66817090e+08]
 [6.63234538e+08]
 [4.60716898e+08]
 [1.08941945e+09]
 [1.54582148e+09]
 [2.47506467e+08]
 [1.43792863e+09]
 [9.90782444e+08]
 [2.40014869e+08]
 [7.52229889e+08]
 [8.13410635e+08]
 [2.70305993e+08]
 [4.60909152e+08]
 [6.94101472e+08]
 [1.12365882e+09]
 [2.43542506e+08]
 [1.63451852e+09]
 [1.61342965e+09]
 [4.690685